# Entity Extraction

In [87]:
import pandas as pd
import spacy 
import ast
import re
from spacy.matcher import PhraseMatcher

In [57]:
# load the data
student_info = pd.read_csv('../data/00 - mock_student_data_final.csv')
module_info = pd.read_csv('../data/01 - mock_module_info.csv')
department_info = pd.read_csv('../data/02 - mock_department_list.csv')
staff_info = pd.read_csv('../data/03 - mock_staff_info.csv')

### Entity extraction functions

#### Extraction from structured column data

In [58]:
def extract_entities(text, label):
    # Extract entities as tuples of (entity text, entity label)
    entities = [(text, label)]
    return entities

#### Extraction from column with list of multiple data

In [59]:
def extract_module_entities(module_list):
    module_list = ast.literal_eval(module_list)
    return [(module, 'MODULE') for module in module_list]

#### Extraction from chunk of unstructured text data

In [61]:
nlp = spacy.load("en_core_web_md")
jobs_and_skills_df = pd.read_csv("jobs_and_skillset_linkedin_final.csv")
jobs_and_skills_df.head()

,Job Title,Skills
0,Academic Advisor,"['Academic Advising', 'Higher Education', 'Stu..."
1,Academic Tutor,"['Tutoring', 'Teaching', 'Peer Tutoring', 'Mat..."
2,Account Coordinator,"['Account Management', 'Account Coordination',..."
3,Account Director,"['Account Management', 'Client Services', 'Cli..."
4,Account Executive,"['Software as a Service (SaaS)', 'Account Mana..."


In [85]:
# get list of job titles and skills 
job_titles = jobs_and_skills_df["Job Title"].to_list()
skills = jobs_and_skills_df["Skills"].unique().tolist()

for i in range(len(skills)):
    skills[i] = ast.literal_eval(skills[i])

# Create a list of unique skills
unique_skills = list(set([skill for sublist in skills for skill in sublist]))

# remove "(programming language)" from  skills and the space before it
unique_skills = [re.sub(r'\s\(.*\)', '', skill) for skill in unique_skills]
print(unique_skills)
print(len(unique_skills))

['Programming', 'Account Management', 'Inventory Planning', 'Facials', 'Continuous Improvement', 'Medicine', 'Autodesk Inventor', 'Supply Chain Management', 'Exercise Prescription', 'Lesson Planning', 'Financial Advisory', 'Organizational Development', 'Multitasking', 'Spanish Teaching', 'Design Engineering', 'Qualitative Research', 'Land Development', 'Web Testing', 'Technical Documentation', 'Skin Care Products', 'Real Estate Negotiating', 'Executive Management', 'Psychological Testing', 'Graphic Arts', 'Information Technology', 'Petroleum', 'Admissions Counseling', 'Knitting', 'Conventional Financing', 'Storytelling', 'Product Demonstration', 'Video Production', 'iOS', 'Care Planning', 'Swim Lessons', 'Quantitative Research', 'Personal Injury Law', 'Retirement Benefits', 'Claim Investigation', 'Material Handling', 'Venture Capital', 'Program Coordination', 'Hospital Pharmacy', 'Supply Chain Optimization', 'Salesforce.com Administration', 'Dental Prophylaxis', 'International Manageme

In [88]:
# Create a spaCy PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

# Convert each skill to a spaCy Doc object and add them to the matcher
patterns = [nlp(skill) for skill in unique_skills]
matcher.add("SKILLS", patterns)

In [145]:
def spacy_extract_entities(text):
    
    # Process text with spaCy model to extract named entities
    doc = nlp(text)

    # Initialize a dictionary to store the entities
    entities = {
        "Job Title": [],
        "Skill": []
    }

    # Extract job titles dynamically using the dynamic list
    for job_title in job_titles:
        if job_title.lower() in text.lower():
            entities["Job Title"].append(job_title)
    
    # Use the matcher to find skills in the text

    for skill in unique_skills:
        # Create a regex pattern with word boundaries around the phrase
        pattern = r"\b" + re.escape(skill) + r"\b"
    
        # Search for the phrase in the text (case-insensitive)
        if re.search(pattern, text, re.IGNORECASE):
            entities["Skill"].append(skill)

    return entities

In [146]:
# Test the dynamic extraction function
sample_text = "John Doe is a Software Engineer at Google with experience in Python and cloud computing. Jane Smith is a Project Manager at Microsoft. She specializes in Project Management and Data Science. Both have worked extensively on Agile and DevOps projects."

# Call the function
entities = spacy_extract_entities(sample_text)

# Print the extracted entities
for entity_type, entity_list in entities.items():
    print(f"{entity_type}: {', '.join(entity_list) if entity_list else 'None'}")

Job Title: Project Manager, Software Engineer
Skill: Management, DevOps, Cloud Computing, Project Management, Python, Data Science


## 00 - Student Data

In [147]:
# check the first few rows of the student_info dataframe
student_info.head()

,Student_Name,Matric_Number,NRIC,Year,Faculties,Major,Second Major,Modules_Completed,Grades,Student_entities,Faculty_entities,Major_entities,Module_entities
0,Tracy Lewis,A0216920B,XXXXX506Z,1,NUS Business School,Business Administration,NaN,"['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...","{'ACC1701B': 'B', 'DMB1202ACC': 'A', 'DMB1201M...","[(Tracy Lewis, STUDENT)]","[(NUS Business School, FACULTY)]","[(Business Administration, MAJOR)]","[(ACC1701B, MODULE), (DMB1202ACC, MODULE), (DM..."
1,Andrew Holden,A0225069H,XXXXX799Z,3,YST Conservatory of Music,Music,NaN,"['CFA1111A', 'MUA1190', 'MUA2109', 'MUA1172', ...","{'CFA1111A': 'F', 'MUA1190': 'A', 'MUA2109': '...","[(Andrew Holden, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(CFA1111A, MODULE), (MUA1190, MODULE), (MUA21..."
2,Phillip Bullock,A0228204E,XXXXX194Z,1,College of Design and Engineering,Electrical Engineering,NaN,"['ME1102', 'BN1111', 'PF1103', 'CN1101A', 'ID1...","{'ME1102': 'F', 'BN1111': 'B-', 'PF1103': 'C+'...","[(Phillip Bullock, STUDENT)]","[(College of Design and Engineering, FACULTY)]","[(Electrical Engineering, MAJOR)]","[(ME1102, MODULE), (BN1111, MODULE), (PF1103, ..."
3,Valerie Rivera,A0200778Y,XXXXX150Z,3,Computing,Business Analytics,NaN,"['CS3236R', 'CS1010', 'CP3209', 'CS1010R', 'IS...","{'CS3236R': 'A+', 'CS1010': 'A+', 'CP3209': 'A...","[(Valerie Rivera, STUDENT)]","[(Computing, FACULTY)]","[(Business Analytics, MAJOR)]","[(CS3236R, MODULE), (CS1010, MODULE), (CP3209,..."
4,Robert Hall,A0222508M,XXXXX824Z,3,YST Conservatory of Music,Music,Business Administration,"['MUA1109', 'CFA1101A', 'MUA2255A', 'MUA1172',...","{'MUA1109': 'C+', 'CFA1101A': 'B-', 'MUA2255A'...","[(Robert Hall, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(MUA1109, MODULE), (CFA1101A, MODULE), (MUA22..."


In [67]:
# extract entities from the student data
student_info['Student_entities'] = student_info['Student_Name'].apply(lambda x: extract_entities(x, 'STUDENT'))
student_info['Faculty_entities'] = student_info['Faculties'].apply(lambda x: extract_entities(x, 'FACULTY'))
student_info['Major_entities'] = student_info['Major'].apply(lambda x: extract_entities(x, 'MAJOR'))
student_info['Module_entities'] = student_info['Modules_Completed'].apply(lambda x: extract_module_entities(x))

In [68]:
#check the first few rows of the student_info dataframe
student_info.head()

,Student_Name,Matric_Number,NRIC,Year,Faculties,Major,Second Major,Modules_Completed,Grades,Student_entities,Faculty_entities,Major_entities,Module_entities
0,Tracy Lewis,A0216920B,XXXXX506Z,1,NUS Business School,Business Administration,NaN,"['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...","{'ACC1701B': 'B', 'DMB1202ACC': 'A', 'DMB1201M...","[(Tracy Lewis, STUDENT)]","[(NUS Business School, FACULTY)]","[(Business Administration, MAJOR)]","[(ACC1701B, MODULE), (DMB1202ACC, MODULE), (DM..."
1,Andrew Holden,A0225069H,XXXXX799Z,3,YST Conservatory of Music,Music,NaN,"['CFA1111A', 'MUA1190', 'MUA2109', 'MUA1172', ...","{'CFA1111A': 'F', 'MUA1190': 'A', 'MUA2109': '...","[(Andrew Holden, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(CFA1111A, MODULE), (MUA1190, MODULE), (MUA21..."
2,Phillip Bullock,A0228204E,XXXXX194Z,1,College of Design and Engineering,Electrical Engineering,NaN,"['ME1102', 'BN1111', 'PF1103', 'CN1101A', 'ID1...","{'ME1102': 'F', 'BN1111': 'B-', 'PF1103': 'C+'...","[(Phillip Bullock, STUDENT)]","[(College of Design and Engineering, FACULTY)]","[(Electrical Engineering, MAJOR)]","[(ME1102, MODULE), (BN1111, MODULE), (PF1103, ..."
3,Valerie Rivera,A0200778Y,XXXXX150Z,3,Computing,Business Analytics,NaN,"['CS3236R', 'CS1010', 'CP3209', 'CS1010R', 'IS...","{'CS3236R': 'A+', 'CS1010': 'A+', 'CP3209': 'A...","[(Valerie Rivera, STUDENT)]","[(Computing, FACULTY)]","[(Business Analytics, MAJOR)]","[(CS3236R, MODULE), (CS1010, MODULE), (CP3209,..."
4,Robert Hall,A0222508M,XXXXX824Z,3,YST Conservatory of Music,Music,Business Administration,"['MUA1109', 'CFA1101A', 'MUA2255A', 'MUA1172',...","{'MUA1109': 'C+', 'CFA1101A': 'B-', 'MUA2255A'...","[(Robert Hall, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(MUA1109, MODULE), (CFA1101A, MODULE), (MUA22..."


In [69]:
#check type of columns in module_info
print(module_info.dtypes)
#change type of description to string
module_info['description'] = module_info['description'].astype(str)

moduleCode                  object
title                       object
description                 object
moduleCredit               float64
department                  object
faculty                     object
gradingBasisDescription     object
prerequisite                object
preclusion                  object
attributes                  object
corequisite                 object
lectureHours               float64
tutorialHours               object
labHours                   float64
projectHours                object
prepHours                  float64
semester_01                float64
semester_02                float64
semester_03                float64
semester_04                float64
UE                           int64
dtype: object


In [96]:
# check first few rows of the module_info dataframe
module_info.head()

,moduleCode,title,description,moduleCredit,department,faculty,gradingBasisDescription,prerequisite,preclusion,attributes,...,prepHours,semester_01,semester_02,semester_03,semester_04,UE,Module_entities,Department_entities,Faculty_entities,Description_entities
0,ABM5001,Leadership in Biomedicine,Leadership is fundamental to the success of in...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,0.0,1.0,0.0,0.0,0,"[(ABM5001, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Person': [], 'Organization': [], 'Location':..."
1,ABM5002,Advanced Biostatistics for Research,This course is served as a concept-based intro...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,0.0,1.0,0.0,0.0,0,"[(ABM5002, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Person': [], 'Organization': ['SPSS'], 'Loca..."
2,ABM5003,Biomedical Innovation & Enterprise,This course will furnish students with a thoro...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,0.0,0.0,0.0,0,"[(ABM5003, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Person': [], 'Organization': [], 'Location':..."
3,ABM5004,Capstone Project,This course encompasses research projects rele...,8.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,1.0,0.0,0.0,0,"[(ABM5004, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Person': [], 'Organization': ['PI'], 'Locati..."
4,ABM5101,Applied Immunology,Advanced immunological applications play impor...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,0.0,0.0,0.0,0,"[(ABM5101, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Person': [], 'Organization': [], 'Location':..."


In [148]:
# extract entities from the module data
module_info['Module_entities'] = module_info['moduleCode'].apply(lambda x: extract_entities(x, 'MODULE'))
module_info['Department_entities'] = module_info['department'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
module_info['Faculty_entities'] = module_info['faculty'].apply(lambda x: extract_entities(x, 'FACULTY'))
module_info['Description_entities'] = module_info['description'].apply(lambda x: spacy_extract_entities(x))

In [149]:
# check first few rows of the module_info dataframe
module_info.head()

,moduleCode,title,description,moduleCredit,department,faculty,gradingBasisDescription,prerequisite,preclusion,attributes,...,prepHours,semester_01,semester_02,semester_03,semester_04,UE,Module_entities,Department_entities,Faculty_entities,Description_entities
0,ABM5001,Leadership in Biomedicine,Leadership is fundamental to the success of in...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,0.0,1.0,0.0,0.0,0,"[(ABM5001, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Job Title': [], 'Skill': ['Leadership']}"
1,ABM5002,Advanced Biostatistics for Research,This course is served as a concept-based intro...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,0.0,1.0,0.0,0.0,0,"[(ABM5002, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Job Title': [], 'Skill': ['Research', 'Extra..."
2,ABM5003,Biomedical Innovation & Enterprise,This course will furnish students with a thoro...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,0.0,0.0,0.0,0,"[(ABM5003, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Job Title': ['Entrepreneur', 'Lecturer'], 'S..."
3,ABM5004,Capstone Project,This course encompasses research projects rele...,8.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,1.0,0.0,0.0,0,"[(ABM5004, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Job Title': [], 'Skill': ['Research']}"
4,ABM5101,Applied Immunology,Advanced immunological applications play impor...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0.0,1.0,0.0,0.0,0.0,0,"[(ABM5101, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]","{'Job Title': [], 'Skill': ['Health', 'Researc..."


In [150]:
# check first few rows of the department_info dataframe
department_info.head()

,department_id,department,faculty
0,NUSDP0001,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine
1,NUSDP0002,Architecture,College of Design and Engineering
2,NUSDP0003,Accounting,NUS Business School
3,NUSDP0004,Communications and New Media,Arts and Social Science
4,NUSDP0005,History,Arts and Social Science


In [151]:
# extract entities from the department data
department_info['Department_entities'] = department_info['department'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
department_info['Faculty_entities'] = department_info['faculty'].apply(lambda x: extract_entities(x, 'FACULTY'))


In [152]:
# check first few rows of the department_info dataframe``
department_info.head()

,department_id,department,faculty,Department_entities,Faculty_entities
0,NUSDP0001,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,"[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
1,NUSDP0002,Architecture,College of Design and Engineering,"[(Architecture, DEPARTMENT)]","[(College of Design and Engineering, FACULTY)]"
2,NUSDP0003,Accounting,NUS Business School,"[(Accounting, DEPARTMENT)]","[(NUS Business School, FACULTY)]"
3,NUSDP0004,Communications and New Media,Arts and Social Science,"[(Communications and New Media, DEPARTMENT)]","[(Arts and Social Science, FACULTY)]"
4,NUSDP0005,History,Arts and Social Science,"[(History, DEPARTMENT)]","[(Arts and Social Science, FACULTY)]"


In [153]:
# check first few rows of the staff_info dataframe
staff_info.head()

,Employee ID,Employee Name,NRIC,DOB,DOJ,Department_ID,Modules Taught
0,NUSPF12345,Marin Sergio Hernandez,XXXXX479A,1983-02-23T00:00:00Z,2009-10-31T00:00:00Z,Electrical and Computer Engineering,CEG5003
1,NUSPF12346,Kathryn Cordova,XXXXX815A,1985-09-02T00:00:00Z,2009-06-07T00:00:00Z,Civil and Environmental Engineering,ESE2102
2,NUSPF12347,Barbara Sanchez,XXXXX777A,1971-07-30T00:00:00Z,2008-05-09T00:00:00Z,Centre for Language Studies,LAT4201HM
3,NUSPF12348,Bryce Lucas,XXXXX610A,1973-07-20T00:00:00Z,2002-01-17T00:00:00Z,BIZ Dean's Office,DMB1203MNO
4,NUSPF12349,Judith Camacho,XXXXX629A,1991-11-16T00:00:00Z,2000-02-13T00:00:00Z,Economics,EC4401HM


In [154]:
# extract entities from the staff_info data
staff_info['Staff_entities'] = staff_info['Employee Name'].apply(lambda x: extract_entities(x, 'STAFF'))
staff_info['Department_entities'] = staff_info['Department_ID'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
staff_info['Module_entities'] = staff_info['Modules Taught'].apply(lambda x: extract_entities(x, 'MODULE'))

In [155]:
# check first few rows of the staff_info dataframe
staff_info.head()

,Employee ID,Employee Name,NRIC,DOB,DOJ,Department_ID,Modules Taught,Staff_entities,Department_entities,Module_entities
0,NUSPF12345,Marin Sergio Hernandez,XXXXX479A,1983-02-23T00:00:00Z,2009-10-31T00:00:00Z,Electrical and Computer Engineering,CEG5003,"[(Marin Sergio Hernandez, STAFF)]","[(Electrical and Computer Engineering, DEPARTM...","[(CEG5003, MODULE)]"
1,NUSPF12346,Kathryn Cordova,XXXXX815A,1985-09-02T00:00:00Z,2009-06-07T00:00:00Z,Civil and Environmental Engineering,ESE2102,"[(Kathryn Cordova, STAFF)]","[(Civil and Environmental Engineering, DEPARTM...","[(ESE2102, MODULE)]"
2,NUSPF12347,Barbara Sanchez,XXXXX777A,1971-07-30T00:00:00Z,2008-05-09T00:00:00Z,Centre for Language Studies,LAT4201HM,"[(Barbara Sanchez, STAFF)]","[(Centre for Language Studies, DEPARTMENT)]","[(LAT4201HM, MODULE)]"
3,NUSPF12348,Bryce Lucas,XXXXX610A,1973-07-20T00:00:00Z,2002-01-17T00:00:00Z,BIZ Dean's Office,DMB1203MNO,"[(Bryce Lucas, STAFF)]","[(BIZ Dean's Office, DEPARTMENT)]","[(DMB1203MNO, MODULE)]"
4,NUSPF12349,Judith Camacho,XXXXX629A,1991-11-16T00:00:00Z,2000-02-13T00:00:00Z,Economics,EC4401HM,"[(Judith Camacho, STAFF)]","[(Economics, DEPARTMENT)]","[(EC4401HM, MODULE)]"


In [159]:
# write the data to new csv files
student_info.to_csv('student_info_final.csv', index=False)
module_info.to_csv('module_info_final.csv', index=False)
department_info.to_csv('department_info_final.csv', index=False)
staff_info.to_csv('staff_info_final.csv', index=False)